In [ ]:
#Install & Imports
!pip -q install transformers datasets accelerate evaluate scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00


In [ ]:
#Upload your CSV
from google.colab import files
files.upload()

Saving tourism_multilingual_1500_FINAL.csv to tourism_multilingual_1500_FINAL.csv


{'tourism_multilingual_1500_FINAL.csv': b'\xef\xbb\xbfid,category,language,question,answer\n1,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,What types of rooms are available at this accommodation?,"We offer single, double, family, and deluxe rooms depending on your preference."\n2,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,What documents do I need to check in?,A passport or national ID card is required for registration.\n3,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Does the room have hot water?,"Yes, hot water is available 24 hours a day."\n4,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Are electrical outlets suitable for charging mobile phones?,"Yes, you can charge your mobile devices using the standard outlets provided in the room."\n5,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Is food available at the accommodation?,"Yes, we provide breakfast, lunch, dinner, and a selection of local meals."\n6,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Is daily room cleaning included?,"

In [ ]:
#Load the FINAL CSV
df = pd.read_csv("tourism_multilingual_1500_FINAL.csv", encoding="utf-8")
df = df.dropna(subset=["category","language","question","answer"]).copy()

print("Shape:", df.shape)
print(df["language"].value_counts())
print("Num categories:", df["category"].nunique())


Shape: (1500, 5)
language
English    500
Sinhala    500
Tamil      500
Name: count, dtype: int64
Num categories: 10


In [ ]:
#Prepare labels + Split (Train / Val / Test)

label_enc = LabelEncoder()
df["label"] = label_enc.fit_transform(df["category"])

# 80% train+val , 20% test
trainval_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)

# From 80%, take 10% as validation → (72% train, 8% val, 20% test)
train_df, val_df = train_test_split(
    trainval_df, test_size=0.1, random_state=42, stratify=trainval_df["label"]
)

print("Train:", train_df.shape)
print("Validation:", val_df.shape)
print("Test:", test_df.shape)


Train: (1080, 6)
Validation: (120, 6)
Test: (300, 6)


In [ ]:
##Convert to HF Dataset + Tokenize
from transformers import AutoTokenizer

model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_ds = Dataset.from_pandas(train_df[["question","label"]].reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df[["question","label"]].reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df[["question","label"]].reset_index(drop=True))

def tok(batch):
    return tokenizer(batch["question"],
                     truncation=True,
                     padding="max_length",
                     max_length=128)

train_tok = train_ds.map(tok, batched=True)
val_tok   = val_ds.map(tok, batched=True)
test_tok  = test_ds.map(tok, batched=True)

train_tok = train_tok.remove_columns(["question"])
val_tok   = val_tok.remove_columns(["question"])
test_tok  = test_tok.remove_columns(["question"])

train_tok.set_format("torch")
val_tok.set_format("torch")
test_tok.set_format("torch")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
train_ds = Dataset.from_pandas(train_df[["question","label"]].reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df[["question","label"]].reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df[["question","label"]].reset_index(drop=True))

def tok(batch):
    return tokenizer(batch["question"], truncation=True, padding="max_length", max_length=128)

train_tok = train_ds.map(tok, batched=True)
val_tok   = val_ds.map(tok, batched=True)
test_tok  = test_ds.map(tok, batched=True)


Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
#Model + TrainingArguments (Colab-safe)
num_labels = len(label_enc.classes_)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1m = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "macro_f1": f1m}

args = TrainingArguments(
    output_dir="xlmr_out",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=20,
    eval_strategy="epoch",   # IMPORTANT: use eval_strategy (not evaluation_strategy)
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    fp16=torch.cuda.is_available(),
)


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


GPU available: True
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
##Train
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,   # <-- validation set
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-2565776440.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#Final Evaluation
metrics = trainer.evaluate(test_tok)
print("✅ Final Test Eval:", metrics)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: WARNING Invalid choice
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


✅ Final Test Eval: {'eval_loss': 2.339733123779297, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.1, 'eval_macro_f1': 0.01818181818181818, 'eval_runtime': 1.2705, 'eval_samples_per_second': 236.131, 'eval_steps_per_second': 29.91}
